#### Definindo as classes.

In [2]:
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

BATCH_SIZE = 1
CLASSES = ['product']
CLASSES += [str(i) for i in range(80 - len(CLASSES))]

#### Preparação dos dataLoaders para os dados de treino, localizando imagens e rotulos YOLO.

In [3]:
dataset_params = {
    'data_dir': "./SKU110K_fixed",
    'train_images_dir':'images/train',
    'train_labels_dir':'labels/train',
    'val_images_dir':'images/val',
    'val_labels_dir':'labels/val',
    'test_images_dir':'images/test',
    'test_labels_dir':'labels/test',
    'classes': CLASSES
}

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)
#train_data.dataset.dataset_params["image_size"] = 416
#val_data.dataset.dataset_params["image_size"] = 416
#test_data.dataset.dataset_params["image_size"] = 416

Indexing dataset annotations: 100%|███████████████████████████████████████████████| 2920/2920 [00:05<00:00, 574.02it/s]


#### Carrega o modelo yolo_nas_s (o menor modelo) com pesos pré-treinados no conjunto COCO e define a executação do treinamento pela GPU.

In [4]:
import torch
from super_gradients.training import models
from super_gradients.training import Trainer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# DEVICE = 'cpu'
model = models.get('yolo_nas_s', pretrained_weights="coco").to(DEVICE)
trainer = Trainer(experiment_name="SKU110K", ckpt_root_dir="./weights")

#### Configuração dos parâmetros de treinamento:

1. O *Warmup* aumenta a taxa de aprendizado durante 3 épocas.
2. O *Cosine Annealing* reduz gradualmente a taxa de aprendizado ao longo do treinamento.
3. Defini o *Optimizer* com 'Adam' e 'wight_decay' para regularização.
4. O EMA suaviza os pesos durante o treinamento.
5. PPYoloELoss é usada para calcular erros de localização, classificação e detecção.
6. Métrica utilizada é a mAP@0.50


> **OBS:** O modelo foi ajustado pra convergir antes de chegar nas 300 épocas, já que a perda estabiliza e o mAP@0.50 começa a melhorar antes disso. O mAP foi usado porque avalia tanto a localização dos objetos (com IoU ≥ 0.50) quanto a classificação, o que faz mais sentido pra detecção de objetos. Acurácia, por exemplo, não ajuda aqui porque não considera a localização. E, por questão de recursos limitados, reduzi pra 5 épocas pra terminar o treino mais rápido e já ter uma ideia dos resultados.




In [5]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

# MAX_EPOCHS = 300
# MAX_EPOCHS = 100
# MAX_EPOCHS = 50
MAX_EPOCHS = 5


train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50',
    #"gradient_accumulation": 1,
}

In [ ]:
trainer.train(
    model=model, 
    training_params=train_params, 
    train_loader=train_data, 
    valid_loader=val_data
)

C:\Users\igor.goncalves\AppData\Roaming\Python\Python39\site-packages\super_gradients\common\registry\registry.py:72: DeprecationWarning: Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
  warnings.warn(f"Object name `{name}` is now deprecated. Please replace it with `{deprecated_names[name]}`.", DeprecationWarning)
C:\Users\igor.goncalves\AppData\Roaming\Python\Python39\site-packages\super_gradients\training\utils\optimizer_utils.py:108: DeprecationWarning: initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})
  warnings.warn(
